In [2]:
print("-------")

from google.colab import drive
drive.mount('/content/drive')



-------
Mounted at /content/drive


In [5]:

file_path = "/content/drive/MyDrive/Big Data Final Project/Final CSVs"
#file_path = 'https://github.com/emdoehring/FinalProject/tree/9740336a6b27d4dd272346bcb02f115398c65117/Final%20CSVs'

In [4]:
#IMPORTS:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.python.keras.layers import Dense
import os
import time

# Data Preprocessing and Tokenizing

In [6]:
df_lumineers = pd.read_csv('/content/drive/MyDrive/Big Data Final Project/Final CSVs/The Lumineers Lyrics.csv')
print(df_lumineers.to_string())

df_frank_ocean = pd.read_csv('/content/drive/MyDrive/Big Data Final Project/Final CSVs/Frank Ocean Lyrics.csv')
print(df_frank_ocean.to_string())


                               Unnamed: 0         Artist                                                              Album                            Release Date                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [7]:
df_lumineers['Lyrics'].to_csv(r'lumineers_data.txt', header=None, index=None, sep='\n', mode='a')
pd.read_csv('lumineers_data.txt')

df_frank_ocean['Lyrics'].to_csv(r'frank_ocean_data.txt', header=None, index=None, sep='\n', mode='a')
pd.read_csv('frank_ocean_data.txt', on_bad_lines='skip')


,"A tornado flew around my room before you came\nExcuse the mess it made, it usually doesn't rain in\nSouthern California, much like Arizona\nMy eyes don't shed tears, but boy, they pour when\n\nI'm thinkin' 'bout you, ooh no, no, no\nI've been thinkin' bout you, you know, know, know\nI've been thinkin' bout you, do you think about me still?\nDo ya, do ya?\n\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh\n\nNo, I don't like you, I just thought you were cool enough to kick it\nGot a beach house I could sell you in Idaho\nSince you think I don't love you, I just thought you were cute\nThat's why I kissed you\nGot a fighter jet, I don't get to fly it though, I'm lying down\n\nThinkin' 'bout you, ooh no, no, no\nI've been thinkin' bout you, you know, know, know\nI've been thinkin' bout you, do you think about me still?\nDo ya, do ya?\n\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh\n\nYes, of course, I remember, how could I forget How could I forget?) how you feel? How you feel?)\nYou know you were my first time, a new feel\nIt won't ever get old, not in my soul, not in my spirit, keep it alive\nWe'll go down this road 'til it turns from color to black and white\n\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh\nOr do you not think so far ahead?\n'Cause I been thinkin' bout forever, ooh"
0,"\n'Round your city, 'round the clock\nEverybod..."
1,"Poolside convo about your summer last night, o..."
2,I thought that I was dreamin' when you said yo...
3,My guy pretty like a girl\nAnd he got fight st...
4,And the peaches and the mangos\nThat you could...
...,...
147,Every night fucks every day up\nEvery day patc...
148,"Walk away, walk away, walk away, away Ooh)\nAw..."
149,And he got fight stories to tell\nI see both s...
150,"Hand me a towel, I'm dirty dancing\nBy myself,..."


In [8]:
lyrics_path = 'lumineers_data.txt'
lyrics_path_1 = 'frank_ocean_data.txt'

In [9]:
#preprocessing by replacing stop characters, \n and \t with spaces
text = open(lyrics_path, 'rb').read().decode(encoding='utf-8')

stopChars = [',','(',')','.','-','[',']','"']

def preprocessText(text):
  text = text.replace('\n', ' ').replace('\t','')
  processedText = text.lower()
  for char in stopChars:
    processedText = processedText.replace(char,' ')
  return processedText


In [10]:
def WordsToList(lyrics_text):
  word_list = [w for w in lyrics_text.split(' ')]
  word_list = [i for i in word_list if i] #removing empty strings from list
  return word_list

preprocessText(text)

lyrics_path = 'lumineers_data.txt'
lyrics_path_1 = 'frank_ocean_data.txt'
text = open(lyrics_path, 'rb').read().decode(encoding='utf-8')
text = preprocessText(text)
lyrics_words = WordsToList(text)
map(str.strip, lyrics_words)

In [11]:
vocab = sorted(set(lyrics_words))
print('Lyrics length (in words):', len(lyrics_words))
print('Unique words in Lyrics text: {}'.format(len(vocab)))
word2idx = {u: i for i, u in enumerate(vocab)}
idx2words = np.array(vocab)
word_as_int = np.array([word2idx[c] for c in lyrics_words])
# The maximum length sentence we want for a single input in words
seqLength = 10
examples_per_epoch = len(lyrics_words)//(seqLength + 1)

wordDataset = tf.data.Dataset.from_tensor_slices(word_as_int)

sequencesOfWords = wordDataset.batch(seqLength + 1, drop_remainder=True) # generating batches of 10 words each

def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequencesOfWords.map(split_input_target)

Lyrics length (in words): 15455
Unique words in Lyrics text: 1852


In [12]:
BATCH_SIZE = 64 # each batch contains 64 sequences. Each sequence contains 10 words (seqLength)
BUFFER_SIZE = 100 # Number of batches that will be processed concurrently

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in words
vocab_size = len(vocab)
# The embedding dimension
embedding_dim = 256
# Number of GRU units
rnn_units = 1024

def createModel(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = createModel(vocab_size = len(vocab), embedding_dim=embedding_dim, rnn_units=rnn_units, batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           474112    
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 1852)          1898300   
                                                                 
Total params: 6310716 (24.07 MB)
Trainable params: 6310716 (24.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)
# We save training checkpoints to facilitate training procedure
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 20
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

tf.train.latest_checkpoint(checkpoint_dir)
model = createModel(len(vocab), embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Epoch 1/20
153/153 [==============================] - 143s 903ms/step - loss: 5.4534
Epoch 2/20
153/153 [==============================] - 133s 867ms/step - loss: 2.6344
Epoch 3/20
153/153 [==============================] - 134s 869ms/step - loss: 1.0954
Epoch 4/20
153/153 [==============================] - 130s 847ms/step - loss: 0.6227
Epoch 5/20
153/153 [==============================] - 132s 866ms/step - loss: 0.4629
Epoch 6/20
153/153 [==============================] - 128s 838ms/step - loss: 0.3910
Epoch 7/20
153/153 [==============================] - 137s 894ms/step - loss: 0.3539
Epoch 8/20
153/153 [==============================] - 139s 907ms/step - loss: 0.3283
Epoch 9/20
153/153 [==============================] - 140s 915ms/step - loss: 0.3138
Epoch 10/20
153/153 [==============================] - 136s 887ms/step - loss: 0.3007
Epoch 11/20
153/153 [==============================] - 142s 931ms/step - loss: 0.2921
Epoch 12/20
153/153 [==============================] - 145s 950

In [53]:
def generateLyrics(model, startString, temp):
  print("---- Generating lyrics starting with '" + startString + "' ----")
  # Number of words to generate
  num_generate = 300

  # Converting our start string to numbers (vectorizing)
  start_string_list =  [w for w in startString.split(' ')]
  input_eval = [word2idx[s] for s in start_string_list]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []

  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # temp represent how 'conservative' the predictions are.
      # Lower temp leads to more predictable (or correct) text
      predictions = predictions / temp
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)
      text_generated.append(' ' + idx2words[predicted_id])

  return (startString + ''.join(text_generated))

In [56]:
model.save('lyrics_model.keras')

print("Example:")
print(generateLyrics(model, startString=u"brightside", temp=0.8))

while (True):
  print('Enter start string:')
  input_str = input().lower().strip()
  print('Enter temp:')
  temp = float(input())
  print(generateLyrics(model, startString=input_str, temp=temp))
  print('Do you want to continue? (yes/no)')
  user_input = input().lower().strip()
  if user_input != 'yes':
      break


Example:
---- Generating lyrics starting with 'brightside' ----
brightside baby tonight tonight tonight tonight i could see it in the air every word was like smoke from a cigarette you know and you know and you know it's your birthday every word every word and we're singin' along it's alright it's alright it's alright it's your birthday dear it's alright it's alright it's alright it's your birthday dear it's alright it's alright it's your birthday dear it's alright it's alright it's alright it's your birthday dear it's alright it's okay and we're carryin' on it's okay and you know and you know and you know it's your birthday every word every word and we're singin' along it's alright it's alright it's alright it's your birthday dear it's alright it's alright it's alright it's your birthday dear it's alright it's alright it's okay and we're carryin' on it's okay and we're singin' along it's alright it's alright it's alright it's your birthday dear it's alright it's alright it's alright i